# Trabajo SEI
Diseño que cambia el color de la imagen de pantalla conectada a HDMI IN al sacar la imagen por HDMI OUT

In [56]:
from pynq import Overlay
from pynq.lib import AxiGPIO
from pynq.lib.video import *

#Carga el diseño
overlay = Overlay('/home/xilinx/jupyter_notebooks/prueba/video_wrapper.bit')

In [57]:
#Configuramos los switches para decidir si queremos hacer o no el cambio de color
#Accede a la IP 
input_ip = overlay.ip_dict['switches_gpio']

#Accede al canal 1 de la IP GPIO
ipt = AxiGPIO(input_ip).channel1

#Configura el canal como entrada
ipt.setdirection("in")

#Lee el estado de los switches
switches_state = ipt.read()
sw0 = switches_state & 0x1  #Extrae el bit 0
sw1 = (switches_state >> 1) & 0x1  #Extrae el bit 1

print("SW0:", sw0)
print("SW1:", sw1)



SW0: 1
SW1: 1


In [58]:
#Conexión HDMI
hdmi_in = overlay.video.hdmi_in
hdmi_out = overlay.video.hdmi_out

In [59]:
#Calcula el estado de grayscale basado en los switches - determinará si hacemos o no el cambio de color
grayscale = sw0 and sw1

#Escribe el valor en el registro de control de color_0
color_ip = overlay.video.hdmi_in.color_0  # Acceso al bloque HLS
color_ip.write(0x10, int(grayscale))  # Escribe 1 si ambos switches están en 1, 0 de lo contrario


In [60]:
#Para comprobar que la escritura ha sido correcta leemos el valor del registro para verificar lo que se ha escrito
valor_escrito = color_ip.read(0x10) 
print(f"Valor escrito en color_0: {valor_escrito}")  

Valor escrito en color_0: 1


In [61]:
#Configuramos los hdmi y los iniciamos
hdmi_in.configure()
hdmi_out.configure(hdmi_in.mode)

hdmi_in.start()
hdmi_out.start()

El monitor debería encenderse y mostrar una pantalla en negro antes de la siguiente linea. Ejecutando la siguiente pasa la entrada a la salida.

In [62]:
hdmi_in.tie(hdmi_out)

In [63]:
#Cerramos la conexión
hdmi_out.close()
hdmi_in.close()